<a href="https://colab.research.google.com/github/r-dube/CICIDS/blob/main/fj_fcnn_num.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# Load the modules used
import numpy as np
import scipy as sci
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [53]:
# For reproducible results
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [54]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [62]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']

    # del df['telecommuting']
    # del df['has_company_logo']
    # del df['has_questions']
    df['telecommuting'] = df['telecommuting'].astype(np.int8)
    df['has_company_logo'] = df['has_company_logo'].astype(np.int8)
    df['has_questions'] = df['has_questions'].astype(np.int8)

    return df

In [63]:
df = fj_load_df_from_url()
df = fj_txt_only(df)

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [61]:
"""
Use Count Vectorizer to convert text into bag of words
"""
train_text, test_text, train_category, test_category, train_f, test_f = \
  train_test_split(df['text'], df['fraudulent'], df[['telecommuting', 'has_company_logo', 'has_questions']], test_size = 0.15)

cv = CountVectorizer(strip_accents='unicode', lowercase=True, stop_words='english', dtype=np.int8) 
cv_train_sparse = cv.fit_transform(train_text)
cv_train_dense = sci.sparse.csr_matrix.todense(cv_train_sparse)

cv_test_sparse = cv.transform(test_text)
cv_test_dense = sci.sparse.csr_matrix.todense(cv_test_sparse)

print('BOW for cv_train:', cv_train_dense.shape)
print('BOW for cv_test:', cv_test_dense.shape)

cv_train_dense = np.hstack((cv_train_dense, train_f))
cv_test_dense = np.hstack((cv_test_dense, test_f))

print('BOW for cv_train:', cv_train_dense.shape)
print('BOW for cv_test:', cv_test_dense.shape)

BOW for cv_train: (15198, 95808)
BOW for cv_test: (2682, 95808)
BOW for cv_train: (15198, 95811)
BOW for cv_test: (2682, 95811)


In [58]:
"""
Fully connected NN model with two hidden layers 
"""
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = cv_train_dense.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy', tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               9615400   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1010      
_________________________________________________________________
dropout_7 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 9,616,421
Trainable params: 9,616,421
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.fit(cv_train_dense, train_category, epochs = 5)

Epoch 1/5
475/475 [==============================] - 32s 67ms/step - loss: 0.0869 - accuracy: 0.9710 - false_positives_3: 42.0000 - false_negatives_3: 398.0000
Epoch 2/5
475/475 [==============================] - 31s 66ms/step - loss: 0.0198 - accuracy: 0.9934 - false_positives_3: 34.0000 - false_negatives_3: 67.0000
Epoch 3/5
475/475 [==============================] - 31s 64ms/step - loss: 0.0062 - accuracy: 0.9983 - false_positives_3: 8.0000 - false_negatives_3: 18.0000
Epoch 4/5
475/475 [==============================] - 31s 65ms/step - loss: 0.0045 - accuracy: 0.9987 - false_positives_3: 10.0000 - false_negatives_3: 9.0000
Epoch 5/5
475/475 [==============================] - 32s 68ms/step - loss: 0.0036 - accuracy: 0.9993 - false_positives_3: 5.0000 - false_negatives_3: 6.0000


In [60]:
pred = model.predict(cv_test_dense)
pred = np.around(pred, decimals = 0)

acc = accuracy_score(pred, test_category)
f1 = f1_score(pred, test_category)
print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))

Accuracy score: 0.9862 F1 score: 0.8604
